# Databricks <> Graphistry Tutorial: Notebooks & Dashboards on IoT data

This tutorial visualizes a set of sensors by clustering them based on lattitude/longitude and overlaying summary statistics

We show how to load the interactive plots both with Databricks notebook and dashboard modes

Steps:

* Install Graphistry
* Prepare IoT data
* Plot in a notebook
* Plot in a dashboard

d
## Install & connect

In [0]:
# Uncomment and run first time
! pip install graphistry

# Can sometimes help:
# dbutils.library.restartPython()

In [0]:
#Optional: Uncomment - We find this speeds up calls 10%+ on some datasets
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
import graphistry  # if not yet available, install and/or restart Python kernel using the above

graphistry.register(
    api=3, username='MY_USERNAME', password='MY_PASSWRD',

    server='hub.graphistry.com'  # or your private server
    protocol='https',  # if http-only, browsers may prevent embedding plots: switch to ".plot(render=False)"
)    

graphistry.__version__

Out[2]: '0.20.2+7.g2ea108549'

## Prepare IoT data
Sample data provided by Databricks

We create tables for different plots:

* Raw table of device sensor reads
* Summarized table:
  - rounded latitude/longitude
  - summarize min/max/avg for battery_level, c02_level, humidity, timestamp

In [0]:
# Load the data from its source.
devices = spark.read \
  .format('json') \
  .load('/databricks-datasets/iot/iot_devices.json')

# Show the results.
print('type: ', str(type(devices)))
display(devices)

type: <class 'pyspark.sql.dataframe.DataFrame'>

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122
3,1210,US,USA,United States,6,sensor-pad-6al7RTAobR,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122
3,1129,CN,CHN,China,7,meter-gauge-7GeDoanM,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123
0,1536,JP,JPN,Japan,8,sensor-pad-8xUD6pzsQI,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123
3,807,JP,JPN,Japan,9,device-mac-9GcjZ2pw,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124
7,1470,US,USA,United States,10,sensor-pad-10BsywSYUF,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import concat_ws, col, round

devices_with_rounded_locations = (
    devices
    .withColumn(
        'location_rounded1',
        concat_ws(
            '_',
            round(col('latitude'), 0).cast('integer'),
            round(col('longitude'), 0).cast('integer')))
    .withColumn(
        'location_rounded2',
        concat_ws(
            '_',
            round(col('latitude'), -1).cast('integer'),
            round(col('longitude'), -1).cast('integer')))
)

cols = ['battery_level', 'c02_level', 'humidity', 'timestamp']
id_cols = ['cca2', 'cca3', 'cn', 'device_name', 'ip', 'location_rounded1', 'location_rounded2']
devices_summarized = (
    devices_with_rounded_locations.groupby('device_id').agg(
        *[F.min(col) for col in cols],
        *[F.max(col) for col in cols],
        *[F.avg(col) for col in cols],
        *[F.first(col) for col in id_cols]
    )
)

# [(from1, to1), ...]
renames = (
    [('device_id', 'device_id')]
    + [(f'first({col})', f'{col}') for col in id_cols]
    + [(f'min({col})', f'{col}_min') for col in cols] 
    + [(f'max({col})', f'{col}_max') for col in cols]
    + [(f'avg({col})', f'{col}_avg') for col in cols]
 )
devices_summarized = devices_summarized.select(list(
       map(lambda old,new:F.col(old).alias(new),*zip(*renames))
       ))

devices_summarized.show()

+---------+----+----+-------------+--------------------+--------------+-----------------+-----------------+-----------------+-------------+------------+-------------+-----------------+-------------+------------+-------------+-----------------+-------------+------------+-----------------+
device_id|cca2|cca3| cn| device_name| ip|location_rounded1|location_rounded2|battery_level_min|c02_level_min|humidity_min|timestamp_min|battery_level_max|c02_level_max|humidity_max|timestamp_max|battery_level_avg|c02_level_avg|humidity_avg| timestamp_avg|
+---------+----+----+-------------+--------------------+--------------+-----------------+-----------------+-----------------+-------------+------------+-------------+-----------------+-------------+------------+-------------+-----------------+-------------+------------+-----------------+
 6| US| USA|United States|sensor-pad-6al7RT...|204.116.105.67| 36_-85| 40_-90| 3| 1210| 51|1458444054122| 3| 1210| 51|1458444054122| 3.0| 1210.0| 51.0|1.458444054122E12|
 7| CN| CHN| China|meter-gauge-7GeDoanM| 220.173.179.1| 23_108| 20_110| 3| 1129| 26|1458444054123| 3| 1129| 26|1458444054123| 3.0| 1129.0| 26.0|1.458444054123E12|
 19| US| USA|United States|meter-gauge-19eg1...|64.124.180.215| 38_-97| 40_-100| 9| 1531| 75|1458444054130| 9| 1531| 75|1458444054130| 9.0| 1531.0| 75.0| 1.45844405413E12|
 22| JP| JPN| Japan| sensor-pad-22oWV2D|221.113.129.83| 36_140| 40_140| 7| 1522| 58|1458444054132| 7| 1522| 58|1458444054132| 7.0| 1522.0| 58.0|1.458444054132E12|
 25| US| USA|United States|therm-stick-25kK6...| 24.154.45.90| 41_-81| 40_-80| 4| 880| 78|1458444054134| 4| 880| 78|1458444054134| 4.0| 880.0| 78.0|1.458444054134E12|
 26| JP| JPN| Japan|sensor-pad-26rAyC...|210.158.147.11| 36_140| 40_140| 9| 1300| 27|1458444054135| 9| 1300| 27|1458444054135| 9.0| 1300.0| 27.0|1.458444054135E12|
 29| NL| NLD| Netherlands|meter-gauge-29lyN...| 83.98.224.49| 52_5| 50_0| 6| 1095| 69|1458444054137| 6| 1095| 69|1458444054137| 6.0| 1095.0| 69.0|1.458444054137E12|
 31| ES| ESP| Spain|meter-gauge-31bcR...| 80.251.64.1| 40_-4| 40_0| 5| 979| 48|1458444054138| 5| 979| 48|1458444054138| 5.0| 979.0| 48.0|1.458444054138E12|
 32| US| USA|United States| sensor-pad-329KFvY| 128.83.9.146| 30_-98| 30_-100| 8| 1262| 30|1458444054139| 8| 1262| 30|1458444054139| 8.0| 1262.0| 30.0|1.458444054139E12|
 34| RU| RUS| Russia|sensor-pad-34F1Ju...| 80.255.179.78| 58_40| 60_40| 7| 940| 89|1458444054140| 7| 940| 89|1458444054140| 7.0| 940.0| 89.0| 1.45844405414E12|
 39| CN| CHN| China|device-mac-39iklY...| 218.7.15.1| 46_127| 50_130| 2| 908| 84|1458444054144| 2| 908| 84|1458444054144| 2.0| 908.0| 84.0|1.458444054144E12|
 43| ES| ESP| Spain|meter-gauge-43RYo...| 80.81.64.142| 40_-4| 40_0| 2| 913| 39|1458444054148| 2| 913| 39|1458444054148| 2.0| 913.0| 39.0|1.458444054148E12|
 50| DE| DEU| Germany| sensor-pad-50g2ukc| 62.153.177.73| 51_9| 50_10| 6| 1217| 69|1458444054152| 6| 1217| 69|1458444054152| 6.0| 1217.0| 69.0|1.458444054152E12|
 54| US| USA|United States|sensor-pad-5410CW...| 204.15.64.249| 33_-117| 30_-120| 8| 1436| 73|1458444054155| 8| 1436| 73|1458444054155| 8.0| 1436.0| 73.0|1.458444054155E12|
 57| US| USA|United States|device-mac-57tRB0...| 64.246.146.13| 43_-74| 40_-70| 6| 1503| 40|1458444054158| 6| 1503| 40|1458444054158| 6.0| 1503.0| 40.0|1.458444054158E12|
 58| MX| MEX| Mexico| sensor-pad-58HgZVw0| 200.76.250.4| 33_-117| 30_-120| 4| 1105| 96|1458444054159| 4| 1105| 96|1458444054159| 4.0| 1105.0| 96.0|1.458444054159E12|
 65| CL| CHL| Chile|therm-stick-65V4W...| 64.76.146.154| -33_-71| -30_-70| 3| 872| 28|1458444054163| 3| 872| 28|1458444054163| 3.0| 872.0| 28.0|1.458444054163E12|
 68| US| USA|United States|sensor-pad-682gym...| 216.83.68.103| 42_-114| 40_-110| 5| 1250| 81|1458444054164| 5| 1250| 81|1458444054164| 5.0| 1250.0| 81.0|1.458444054164E12|
 71| BY| BLR| Belarus|meter-gauge-712Jg...| 195.222.71.1| 54_28| 50_30| 8| 934| 43|1458444054166| 8| 934| 43|1458444054166| 8.0| 934.0| 43.0|1.458444054166E12|
 72| FI| FIN| Finland| sensor-pad

## Notebook plot

* Simple: Graph connections between `device_name` and `cca3` (country code)
* Advanced: Graph multiple connections, like `ip -> device_name` and `locaation_rounded1 -> ip`

In [0]:
displayHTML(graphistry.edges(devices.sample(fraction=0.1), 'device_name', 'cca3').plot())

<iframe id="f27bbb14-b8f2-46db-a06e-58511ab4cb06" src="https://hub.graphistry.com/graph/graph.html?dataset=c3736ad960b7404abb492c3951645e25&type=arrow&viztoken=7d98e0b9-52d8-4e95-904d-01323b77bbfe&usertag=0c107493-pygraphistry-0.20.2+7.g2ea108549&splashAfter=1637539061&info=true"
 allowfullscreen="true" webkitallowfullscreen="true" mozallowfullscreen="true"
 oallowfullscreen="true" msallowfullscreen="true"
 style="width:100%; height:500px; border: 1px solid #DDD; overflow: hidden"
 
 >

In [0]:
hg = graphistry.hypergraph(
    devices_with_rounded_locations.sample(fraction=0.1).toPandas(),
    ['ip', 'device_name', 'location_rounded1', 'location_rounded2', 'cca3'],
    direct=True,
    opts={
        'EDGES': {
            'ip': ['device_name'],
            'location_rounded1': ['ip'],
            'location_rounded2': ['ip'],
            'cca3': ['location_rounded2']
        }
    })
displayHTML(hg['graph'].plot())

# links 78948
# events 19737
# attrib entities 41079

<iframe id="5b117aa1-a78d-4a84-b2b7-8c7bea0ba39b" src="https://hub.graphistry.com/graph/graph.html?dataset=4d948abc665144a28927737ce4479cc1&type=arrow&viztoken=ac9ad101-cc65-4ca1-9640-2f71e90172a7&usertag=0c107493-pygraphistry-0.20.2+7.g2ea108549&splashAfter=1637539012&info=true"
 allowfullscreen="true" webkitallowfullscreen="true" mozallowfullscreen="true"
 oallowfullscreen="true" msallowfullscreen="true"
 style="width:100%; height:500px; border: 1px solid #DDD; overflow: hidden"
 
 >

## Dashboard plot

* Make a `graphistry` object as usual...
* ... Then disable the splash screen and optionally set custom dimensions

The visualization will now load without needing to interact in the dashboard (`view` -> `+ New Dashboard`)

In [0]:
g = graphistry.edges(devices.sample(fraction=0.1), 'device_name', 'cca3')

displayHTML(
    g
        .settings(url_params={'splashAfter': 'false'})
        .plot(override_html_style="""
            border: 1px #DDD dotted;
            width: 50em; height: 50em;
        """)
)

<iframe id="2d794f13-2760-44ff-8297-3c662eaab2d4" src="https://hub.graphistry.com/graph/graph.html?dataset=f24ffcf3237641aa880a711fb7e18720&type=arrow&viztoken=b4fa60c5-eba4-4e4e-a182-8e3ec6ce4c87&usertag=0c107493-pygraphistry-0.20.2+7.g2ea108549&splashAfter=1637539411&info=true&splashAfter=false"
 allowfullscreen="true" webkitallowfullscreen="true" mozallowfullscreen="true"
 oallowfullscreen="true" msallowfullscreen="true"
 style="
 border: 1px #DDD dotted;
 width: 50em; height: 50em;
 "
 
 >